<a href="https://colab.research.google.com/github/dbremont/models/blob/main/models/Boston_Housing_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ...

## References

- [The Boston Housing Dataset](https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset)